# Python 知识
## import 导入链的注意事项

导入是将被导入模块中的名称添加到导入模块的命名空间中的过程。

假设当前模块名为 `M`，`from X import Xo` 会将 `Xo` 直接添加到当前模块的命名空间：可以访问 `M.Xo`

而 `import X` 会将整个模块 `X` 作为一个对象添加到当前模块的命名空间：`M.X`（包括例如 `M.X.Xo`）等。

例子 1：

```python
# 模块A (a.py)
value = 100
def func(): 
    return "A.func"

# 模块B (b.py)
from A import value, func  # 从A导入特定内容到B的命名空间

# 模块C (c.py)
import B

# 在C中可以使用:
print(B.value)  # 可以访问，输出100
print(B.func())  # 可以访问，输出"A.func"
```
例子 2：
```python
# 模块A (a.py)
value = 100
def func(): 
    return "A.func"

# 模块B (b.py)
import A  # A成为B模块命名空间中的一个对象

# 模块C (c.py)
import B

# 在C中:
print(B.A.value)  # 可以访问，输出100
print(B.value)    # 错误! B中没有直接定义value

## `with open(str, '*') as f:`
等价于
```python
try:
    f = open(str, '*')
    ...
finally:
    if f:
        f.close()
```

## `__new__(cls, *args, **kwargs)`

### `__new__(cls, *args, **kwargs)` 是静态方法
`__new__` 主要作用是分配内存并创建实例本身，其一般是通知父类的 `__new__` 创建对应类型的实例：
```python
def __new__(cls, *args, **kwargs):
    return super().__new__(cls)
```
如果父类直接继承自 `object` 或者没有重定义 `__new__`，那么最终实际上是 `object` 调用其 `__new__` 创建对应类型的实例。

由于 `__new__(cls, *args, **kwargs)` 不是类方法，所以 `cls` 是手动传入的，其作用在于告诉 `object` 要创建什么类型的实例。
### 类型创建实例的过程
当一个类型创建实例时，例如 `instance = MyClass(*args, **kwargs)`，Python 内部实际执行
```python
instance = MyClass.__new__(MyClass, *args, **kwargs)
if isinstance(instance, MyClass):
    instance.__init__(*args, **kwargs)
```
`instance = MyClass.__new__(MyClass, *args, **kwargs)` 这一行相当于告诉 `object` 创建一个 `MyClass` 类型的实例，而 `instance.__init__(*args, **kwargs)` 负责初始化。

## Python 类型和实例的内存构成
### 实例
`__dict__` 字典的引用：存储实例在运行时动态添加或在 `__init__` 方法中设置的属性。
- 如果一个类定义了 `__slots__`，并且其中没有包含 `__dict__`，那么该类的实例将不会有 `__dict__` 属性。这种情况下，实例属性（名称包含在 `__slots__` 中）在内存中是独立存储的
- 许多内置类型的实例（如 list, tuple, str, int, set, map 对象，甚至 dict 实例本身）为了效率和设计目的，没有 `__dict__` 属性，属性直接存储在固定内存布局中

`__class__` 引用：指向实例所属的类对象（如 `<class '__main__.A'>`）

`__weakref__`：支持弱引用机制，仅在需要时存在。
### 类型
`__dict__` 字典的引用：存储类属性、方法、描述符等。

`__name__`：类的名称。

`__module__`：类定义所在的模块名。

`__bases__`：类的父类元组

`__doc__`：类的文档字符串。

`__qualname__ `：类的限定名称（包含嵌套类路径）。

`方法表`：存储类的方法（如 `__init__`, `__str__`）。

# 代码设计

## 类继承关系
```mermaid
classDiagram
    %% 基本类
    class object
    
    %% 一级继承
    class dict
    class Default
    class Pickleable
    class Documented
    class tuple
    
    %% 二级继承
    class atomic_dict
    class PickleableDict
    class Configured
    class namedtuple
    
    %% 三级继承
    class Config
    class DumpTuple
    
    %% 四级继承
    class AtomicConfig
    
    %% 所有类都继承自 object
    object <|-- dict
    object <|-- Default
    object <|-- Pickleable
    object <|-- Documented
    object <|-- tuple
    
    %% 其他继承关系
    dict <|-- atomic_dict
    tuple <|-- namedtuple
    
    Pickleable <|-- PickleableDict
    dict <|-- PickleableDict
    
    Pickleable <|-- Configured
    Documented <|-- Configured
    
    PickleableDict <|-- Config
    Documented <|-- Config
    
    Config <|-- AtomicConfig
    atomic_dict <|-- AtomicConfig
    
    namedtuple <|-- DumpTuple
```
`atomic_dict` 和 `Config` 都是 `dict` 的子类，它们三个类的 `items()` 都是用来存储`配置项`的。

`Config` 的 `__dict__` 用来存储 `如何管理配置项的项`。但是，也可以通过 `_as_attrs_` 来将`配置参数`同时存储到 `__dict__`。

`Configured` 的设计思想：一个复杂对象的状态由两部分组成：
- （1）配置 `_config`：由 Config 实例管理的一系列参数，这些参数通常在对象创建时设置，并控制对象的行为。
- （2）其他可写属性 `writeable_attrs`：对象在生命周期中可能会改变的内部状态，这些状态不适合放在配置字典里。比如当前的持仓量、交易次数。

### `Pickleable` 和 `PickleableDict`
涉及到两个序列化库 `pickle` 和 `dill`：
- `pickle`（标准库）：只能序列化 Python 的基本数据类型和部分对象等，对复杂对象类型支持有限。
- `dill`：是 pickle 的扩展，几乎可以序列化所有 Python 对象。
  
`Pickable`：继承该类型的类一般会重写 `dumps` 和 `loads` 方法
- `dumps`：如何将实例序列化为字节流
- `loads`：如何将字节流反序列化为实例
- `save`：使用 `dumps` 序列化为字节流，然后存入到文件中
- `load`：读取文件中的字节流后，使用 `loads` 反序列化为实例

`PickleableDict`：继承自 `Pickable` 和 `dict`，而继承该类型的类一般会重写 `load_update` 方法
- 重写的 `dumps`：
  - 构建一个 `dict`，对于 `items()` 中的每个键值对 `(k: v)`：
      - 如果 `v` 继承了 `Pickeable`：存入 `(k: DumpTuple(v.__class__, v.dumps()))`。
      - 否则：存入 `(k: v)`
  - 然后将该 `dict` 使用 **dill** 序列化为字节流
- 重写的 `loads`：
  - 先使用 **dill** 反序列化字节流
  - 然后对于该 `dict` 的每个 `(k: v)`，如果 `v` 是 `DumpTuple` 类型
    - 替换为 `(k: v.cls.loads(v.dumps))`
  - 然后使用该 `dict` 构造一个当前类的实例。
- `update`：清空　`items()`，然后使用 `load` 从文件中取出字节流并反序列化，然后更新 `items()`

```python
class Pickleable:

    def dumps(self, **kwargs) -> bytes:
        return pickle.dumps(self, protocol=pickle.HIGHEST_PROTOCOL)

    @classmethod
    def loads(cls: tp.Type[PickleableT], dumps: bytes, **kwargs) -> PickleableT:
        return pickle.loads(dumps)

    def save(self, fname: tp.FileName, **kwargs) -> None:
        dumps = self.dumps(**kwargs)
        with open(fname, "wb") as f:
            f.write(dumps)

    @classmethod
    def load(cls: tp.Type[PickleableT], fname: tp.FileName, **kwargs) -> PickleableT:
        with open(fname, "rb") as f:
            dumps = f.read()
        return cls.loads(dumps, **kwargs)

class PickleableDict(Pickleable, dict):
    def dumps(self, **kwargs) -> bytes:
        dct = dict()
        for k, v in self.items():
            if isinstance(v, Pickleable):
                # k: (v.__class__, v.dumps(**kwargs))
                dct[k] = DumpTuple(cls=v.__class__, dumps=v.dumps(**kwargs))    
            else:
                # k: v
                dct[k] = v  
        return dill.dumps(dct, **kwargs)

    @classmethod
    def loads(cls: tp.Type[PickleableDictT], dumps: bytes, **kwargs) -> PickleableDictT:
        config = dill.loads(dumps, **kwargs)
        for k, v in config.items():
            if isinstance(v, DumpTuple):
                config[k] = v.cls.loads(v.dumps, **kwargs)
        return cls(**config)

    def load_update(self, fname: tp.FileName, **kwargs) -> None:
        self.clear()
        self.update(self.load(fname, **kwargs))

### `Documented`
对于一个继承了 `Documented` 的类型 `A`，其应该实现 `to_doc` 方法。

这样，当一个实例 `a` 使用 `str(a)` 时：
- 相当于调用 `a.to_doc()`
- 如果没有实现 `to_doc`，则相当于调用 `a.__repr__()`

```python
class Documented:

    def to_doc(self, **kwargs) -> str:
        raise NotImplementedError

    def __str__(self):
        try:
            return self.to_doc()
        except NotImplementedError:
            return repr(self)
```

### `Config`
`InConfigLikeT` 类型的实例可以认为是一个**配置**。

其图中，属于 `InConfigLikeT` 类型的域中的字段可划分为两个部分：
- `items()`——**配置项**
- `__dict__`——**管理配置项的项**（仅在该域为 `Config` 时才存在）
  - **`_reset_dct_`**: `Union[None, dict]`
  - **`_copy_kwargs_`** 和 **`_reset_dct_copy_kwargs_`**: `Union[None, Dict[str, Any]]`
      - 其键保证一定是类似于 {'copy_mode': 'shallow', 'nested': True}，无后续嵌套。
      - 这两个参数决定拷贝 `Config.copy` 调用时，`dct.items()` 和 `dct.__dict__[_reset_dct_]` 如何被拷贝
  - **`_frozen_keys_`**，**`_readonly_`**，**`_nested_`**，**`_as_attrs_`**: `Optional[bool]`
  - **`_convert_dicts_`**: `Optional[Union[bool, Type["Config"]]]`

对 `InConfigLikeT` 类型的**操作**，沿着图进行
- **操作末端**
  - 非 `InConfigLikeT` 类型的字段
  - `atomic_dict` 类型的字段
  - 某些操作有参数 `nested`，当 `nested == False` 时，所有直接字段都是操作末端

### `AtomicConfig`
继承 `Config` 和 `atomic_dict`，作为**原子配置类**，以便在操作时作为**操作末端**。


## `InConfigLikeT` 的拷贝与合并

### 递归

递归函数是任意的，只要一个函数的实现中调用了自身。

**自顶向下**
- 在域 `D` 上完成某功能，是否能将其转换为：在 `D` 的不相交子域上各自完成的功能然后整合而成。
- 核心在于可以将 `在域 D 某功能` 充分、完备的描述出来。

**自底向上**
- 因为域 `D` 构成过于复杂，事先甚至无法将在域 `D` 上的要实现的功能完备描述出来。
- 可以先考虑域 `D` 的最简单情况：可以直接解决而无须递归。然后逐步考虑完整的情况。

很多问题无法运用套路，但是仍可能通过递归解决，需要具体分析。

### 实例的内存理解
`域` 即一段内存空间
- 每个域具有 `域类型`
- 域分为`值域`和`引用域`
    - 值域存储的是实际内容 `域名.字段名 = 值`
    - 引用域存储的是另一个域的地址 `域名.字段名 = 域名`
- 对于一个域 $o$，从其引用域延申到其它域，这样一直延申，可以得到一个分层次的、由多个域构成、多分支有向连接的 `图`
    - 当然可能有其它域的字段引用 `o`，但是这里只考虑从 `o` 出发延申得到的
    - 图上的两个域 $a$ 和 $b$，如果 $a.x=b$，则称有一条 $a\mathop  \to \limits^{a.x} b$ 的`路径`。
        - 如果存在形如 $a \to  \cdots  \to a$ 的路径，则称该图是 `有环图`
    - 如何不重复的遍历图中的所有域：
        ```python
        def Vist(域x, visited):
            if visited.contains(id(x))
                return
            visited[id(x)] = True
            对于 x.引用域中的每个引用i：
                Vist(i, visited)
        ```

### 浅拷贝和深拷贝
#### 浅拷贝
将被拷贝实例的域中的内容完整的拷贝到另一段内存空间。所以两段内存空间中的引用域指向相同的实例。
#### 深拷贝
深拷贝就是在内存中开辟另一个图，使得值域、对应有向连接和原图相同。

假设有一个函数 `deepcopy(实例x, memo)`，返回深拷贝实例x后的地址，其伪代码如下：
```python
def deepcopy(实例x, memo):
    if memo.contains(id(x))
        return memo[id(x)]
    内存中创建一个和x同类型的实例y
    memo[id(x)] = y
    y.值域 = x.值域
    遍历 x.引用域中的每个引用i：
        y.i = deepcopy(引用i，memo)
    return y
```

### `convert_to_dict`
根据 `dct` 提取一个新的 `dict` 实例，忽略其中属于 `__dict__` 的字段。

```python
def convert_to_dict(dct: InConfigLikeT, nested: bool = True) -> dict:
    if dct is None:
        dct = {}
    if isinstance(dct, atomic_dict):
        dct = atomic_dict(dct)
    else:
        dct = dict(dct) 
    if not nested:
        return dct
    for k, v in dct.items():
        if isinstance(v, dict):
            dct[k] = convert_to_dict(v, nested=nested)
        else:
            dct[k] = v
    return dct
```

### `update_dict`
用 `y` 中不属于 `__dict__` 的字段更新对应的 `x` 中不属于 `__dict__` 的字段。
- `same_keys == True` 即只更新 `x` 中存在的字段，否则会插入
- 这里所谓的**更新**是指函数 `set_dict_item`
```python
def update_dict(x: InConfigLikeT,
                y: InConfigLikeT,
                nested: bool = True,
                force: bool = False,
                same_keys: bool = False) -> None:
    if x is None:
        return
    if y is None:
        return
    checks.assert_instance_of(x, dict)
    checks.assert_instance_of(y, dict)
    for k, v in y.items():
        if nested \
                and k in x \
                and isinstance(x[k], dict) \
                and isinstance(v, dict) \
                and not isinstance(v, atomic_dict):
            update_dict(x[k], v, force=force)
        else:
            if same_keys and k not in x:
                continue
            set_dict_item(x, k, v, force=force)
```

### `Config.__copy__`

如果 `self.as_attrs_ == True`，说明 `配置项` 同时存到了 `__dict__` 中。

所以函数 `_clear_attrs` 的作用就是：
- 如果 `self.as_attrs_ == true`，那么从 `__dict__` 中删除 `prior_keys` 中那些不在 `items()` 中的项。

```python
def  _clear_attrs(self, prior_keys: tp.Iterable[str]) -> None:
    if self.as_attrs_:
        for k in set(prior_keys).difference(self.keys()):
            del self.__dict__[k]
```

函数 `_clear_attrs` 的作用是：
- 如果 (强制 || 该Config非只读) && (强制 || 该Config非冻结键) && (self.as_attrs_ == true) 成立
    - 从管理配置项的项 `__dict__` 中删除属于配置项 `items()` 中的项。

```python
def clear(self, force: bool = False) -> None:
    if not force and self.readonly_:
        raise TypeError("Config is read-only")
    if not force and self.frozen_keys_:
        raise KeyError(f"Config keys are frozen")
    prior_keys = list(self.keys())
    dict.clear(self)
    self._clear_attrs(prior_keys)
```

函数 `__copy__` 的作用是：
- 生成一个新的 Config 实例，其 `items()` 和 `__dict__` 中的每个键值对都与 `self` 相同。
- 但是额外保证了 `items()` 和 `__dict__` 是不相交的。

```python
def __copy__(self: ConfigT) -> ConfigT:
    cls = self.__class__
    self_copy = cls.__new__(cls)
    for k, v in self.__dict__.items():
        if k not in self_copy:  
            self_copy.__dict__[k] = v
    self_copy.clear(force=True)
    self_copy.update(copy(dict(self)), nested=False, force=True)
    return self_copy
```

### `Config.__deepcopy__`
函数 `Config.update` 的作用是：
- 根据参数 args 和 kwargs 构造一个 dict，并用该 dict 去更新 `self` 中对应的不属于 `__dict__` 的字段。
```python
def update(self, *args, nested: tp.Optional[bool] = None, force: bool = False, **kwargs) -> None:
    other = dict(*args, **kwargs)
    if nested is None:
        nested = self.nested_
    update_dict(self, other, nested=nested, force=force)
```

函数 `__deepcopy__` 的作用是：
- 生成一个新的 Config 实例，其 `items()` 和 `__dict__` 中的每个键值对的值都是由 `self` 对应的深拷贝得到的。
- 但是额外保证了 `items()` 和 `__dict__` 是不相交的。

```python
def __deepcopy__(self: ConfigT, memo: tp.DictLike = None) -> ConfigT:
    if memo is None:
        memo = {}
    cls = self.__class__
    self_copy = cls.__new__(cls)
    memo[id(self)] = self_copy
    for k, v in self.__dict__.items():
        if k not in self_copy:  
            self_copy.__dict__[k] = deepcopy(v, memo)
    self_copy.clear(force=True)
    self_copy.update(deepcopy(dict(self), memo), nested=False, force=True)
    return self_copy
```

### `copy_dict` 和 `Config.copy`

首先注意到 `merge_dicts` 被 `Config.copy` 使用仅有如下两句：
- `merge_dicts(self.reset_dct_copy_kwargs_, copy_kwargs, reset_dct_copy_kwargs)`
- `merge_dicts(self.copy_kwargs_, copy_kwargs)`

    根据**递归设计：代码分析**，这就是简单的无嵌套字典 `Dict[str, Any]` 的合并。

函数 `copy_dict(dct, copy_mode, nested)` 实际上是**自底向上**的递归设计：

- 如果 `copy_mode == deep`，直接深拷贝 `dct`。
- 否则
  
    （3.1）**Dict 且无 Config**：
    - 按照 `copy_mode = hybrid/shallow` 来选择是 `copy(*)/*` 来拷贝 `dct` 中的每个操作末端。
  
    （3.2）**Config 且图中无其它 Config**：`(copy_mode, nested)` 作为 `copy_kwargs` 调用 `Config.copy`
    - 生成一个新的 Config 实例，其 `items()` 和 `__dict__` 中的每个键值对都与 `self` 相同。
    - 合并 `self.reset_dct_copy_kwargs_, copy_kwargs` 构成 `(copy_mode, nested)`
  
        使用 （3.1）拷贝 `self.__dict__[_reset_dct_].items()`
    - 合并 `self.copy_kwargs_, copy_kwargs` 构成 `(copy_mode, nested)` 

        使用 （3.1）拷贝 `self.items`
    
    （3.3）显然 `dct` 的一切情况都可由基本情况 （3.1）和 （3.2）构成。

```python
def copy_dict(dct: InConfigLikeT, copy_mode: str = 'shallow', nested: bool = True) -> OutConfigLikeT:
    if dct is None:
        dct = {}
    checks.assert_instance_of(copy_mode, str)
    copy_mode = copy_mode.lower()
    if copy_mode not in ['shallow', 'hybrid', 'deep']:
        raise ValueError(f"Copy mode '{copy_mode}' not supported")

    if copy_mode == 'deep':
        return deepcopy(dct)
    
    if isinstance(dct, Config):
        return dct.copy(
            copy_mode=copy_mode,
            nested=nested
        )
        
    dct_copy = copy(dct) 
    for k, v in dct_copy.items():
        if nested and isinstance(v, dict):
            _v = copy_dict(v, copy_mode=copy_mode, nested=nested)
        else:
            if copy_mode == 'hybrid':
                _v = copy(v)  
            else:
                _v = v
        set_dict_item(dct_copy, k, _v, force=True)
    return dct_copy
```

函数 `Config.copy` 的作用是：
- 生成一个新的 Config 实例，其 `items()` 和 `__dict__` 中的每个键值对都与 `self` 相同。
- 合并 `self.reset_dct_copy_kwargs_, copy_kwargs, reset_dct_copy_kwargs` 构成 `(copy_mode, nested)`

    使用 `copy_dict` 拷贝 `self.__dict__[_reset_dct_].items()`
- 合并 `self.copy_kwargs_, copy_kwargs` 构成 `(copy_mode, nested)` 

    使用 `copy_dict` 拷贝 `self.items`

```python
def copy(self: ConfigT, reset_dct_copy_kwargs: tp.KwargsLike = None, **copy_kwargs) -> ConfigT:
    self_copy = self.__copy__()
    
    reset_dct_copy_kwargs = merge_dicts(self.reset_dct_copy_kwargs_, copy_kwargs, reset_dct_copy_kwargs)
    reset_dct = copy_dict(dict(self.reset_dct_), **reset_dct_copy_kwargs)
    self.__dict__['_reset_dct_'] = reset_dct

    copy_kwargs = merge_dicts(self.copy_kwargs_, copy_kwargs)
    dct = copy_dict(dict(self), **copy_kwargs)
    self_copy.update(dct, nested=False, force=True)

    return self_copy
```

### `merge_dicts`

函数 `merge_dicts` 的作用是：合并 `*dicts`

（1）如果 `to_dict == True`
- 先对 `*dicts` 使用函数 `convert_to_dict` 即提取新的 `dict` 实例，忽略其中属于 `__dict__` 的字段

（2）如果 `copy_mode is not None && (to_dict == False || copy_mode != 'shallow')`
- 使用 `copy_dict` 拷贝 `dicts` 中的每一个
- 如果 `to_dict == True && copy_mode == 'shallow'`，拷贝后的结果和（1）是一样的，所以有 `&&` 后面的这个条件。

（3）使用函数 `update_dict` 将 `*dicts.items()` 合并到一个。

```python
def merge_dicts(*dicts: InConfigLikeT,
                to_dict: bool = True,
                copy_mode: tp.Optional[str] = 'shallow',
                nested: bool = True,
                same_keys: bool = False) -> OutConfigLikeT:
    if to_dict:
        dicts = tuple([convert_to_dict(dct, nested=nested) for dct in dicts])
    if copy_mode is not None:
        if not to_dict or copy_mode != 'shallow':
            dicts = tuple([copy_dict(dct, copy_mode=copy_mode, nested=nested) for dct in dicts])
    x, y = dicts[0], dicts[1]
    should_update = True
    if x.__class__ is dict and y.__class__ is dict and len(x) == 0:
        x = y
        should_update = False
    if isinstance(x, atomic_dict) or isinstance(y, atomic_dict):
        x = y
        should_update = False
    if should_update:
        update_dict(x, y, nested=nested, force=True, same_keys=same_keys)
    if len(dicts) > 2:
        return merge_dicts(
            x, *dicts[2:],
            to_dict=False,  
            copy_mode=None,  
            nested=nested,
            same_keys=same_keys
        )
    return x
```